In [174]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go


## Read In and Clean Data

In [175]:
from data.data import AOTY

In [176]:
AOTY

,Rank,Lister,Artist,Album,album_submission_count,top_10_count_album,album_score,top_10_score_album,album_average_rank,unique_album_submission,artist_album_release_count,multi_album_artist
0,1,Aaron,The Dirty Nil,Fuck Art,2,False,15,False,3.500000,False,1,False
1,2,Aaron,Whitechapel,Kin,1,False,9,False,2.000000,True,1,False
2,3,Aaron,That Handsome Devil,Your Parents Are Sellouts,1,False,8,False,3.000000,True,1,False
3,4,Aaron,Billie Eilish,Happier Than Ever,22,True,137,True,4.772727,False,1,False
4,5,Aaron,Lorna Shore,...And I Return To Nothingness,1,False,6,False,5.000000,True,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
975,6,Zack,Mark Barlow,Hymns & Soul,2,False,15,False,3.500000,False,1,False
976,7,Zack,Aquilo,A Safe Place To Be,1,False,4,False,7.000000,True,1,False
977,8,Zack,Tingsek,Home,2,False,4,False,9.000000,False,1,False
978,9,Zack,Ben Howard,Collections From The Whiteout,2,False,6,False,8.000000,False,1,False


## Get Extra Metadata

* Total Listers
* Total Artists
* Total Albums

In [177]:
from data.data import TOTAL_LISTERS, TOTAL_ARTISTS, TOTAL_ALBUMS

In [178]:
TOTAL_LISTERS

98

In [179]:
TOTAL_ARTISTS

432

In [180]:
TOTAL_ALBUMS

436

* album_submission_count for each album
* unique_album_submission tag for each submission
* artist_album_release_count
* multi_album_artist

In [181]:
len(set(pd.read_csv("data/AOTY-2021-lists.csv")["Album"]))

436

In [182]:
AOTY["Album"].nunique()

436

### Scrape Genre Metadata?

---

# Dataviz

##### Top 10 Album Barchart

In [183]:
from data.data import AOTY_by_album

from data.datafunc import get_albums_of_note

In [184]:
## DYNAMIC COMPONENTS

COLUMN = "album_score"
# COLUMN = "album_submission_count"

USER_ALBUM = "Glow On"


In [185]:
## GLOBALS FOR PLOTTING

APP_COLORS = {
    "accent" : "crimson",
    "standard" : "navy",
    "dim" : "lightgrey",
    "dark" : "dark grey",
    "light" : "antiquewhite"
}

# TODO: APP_FONTS

In [186]:
# so we can change just COLUMN
TOP_COL_LOOKUP = {
    "album_score": "top_10_score_album",
    "album_submission_count": "top_10_count_album",
}


AOTY_by_album = (
    AOTY_by_album.sort_values(COLUMN, ascending=False)
    .reset_index()
    .drop(columns="index")
)

top_10_albums = AOTY_by_album.loc[AOTY_by_album[TOP_COL_LOOKUP[COLUMN]] == True]


album_search = [USER_ALBUM] if USER_ALBUM else None
albums_of_note = get_albums_of_note(AOTY_by_album, album_search)


In [289]:
figure = px.bar(
    top_10_albums,
    x="Album",
    y=["album_score"],
    barmode="group",
    text_auto=True,
    hover_name="Album",
    custom_data=["Artist"],
    title="Album Scores for our Top 10 Albums of 2021",
    labels={"value": "Album Score"},
    template="simple_white",
    color_discrete_sequence=[APP_COLORS["standard"]]

)
figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata}</b> - <i>%{x}</i>",
            "",
            "<extra></extra>",
        ]
    )
)
figure.update_layout(hovermode="closest", showlegend=False)

figure.add_hrect(
    y0=AOTY_by_album["album_score"].quantile(0.25),
    y1=AOTY_by_album["album_score"].quantile(0.75),
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text=f"'Usual' Album scores: {AOTY_by_album['album_score'].quantile(0.25):.2f} - {AOTY_by_album['album_score'].quantile(0.75):.2f}",
    annotation_position="inside right",
    annotation_font_color= APP_COLORS["light"],
)

figure.show()


In [288]:
figure = px.bar(
    top_10_albums,
    x="Album",
    y=["album_submission_count"],
    barmode="group",
    text_auto=True,
    hover_name="Album",
    custom_data=["Artist"],
    title="Album Submission Count for our Top 10 Albums of 2021",
    labels={"value": "Album Submissions"},
    template="simple_white",
    color_discrete_sequence=[APP_COLORS["standard"]]

)
figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata}</b> - <i>%{x}</i>",
            "",
            "<extra></extra>",
        ]
    )
)
figure.update_layout(hovermode="closest", showlegend=False)

figure.add_hrect(
    y0=AOTY_by_album["album_submission_count"].quantile(0.25),
    y1=AOTY_by_album["album_submission_count"].quantile(0.75),
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text=f"'Usual' Submission counts: {AOTY_by_album['album_submission_count'].quantile(0.25):.2f} - {AOTY_by_album['album_submission_count'].quantile(0.75):.2f}",
    annotation_position="inside right",
    annotation_font_color=APP_COLORS["light"],
)

figure.show()


In [189]:
max_submissions = AOTY_by_album["album_submission_count"].max()

figure = px.scatter(
    AOTY_by_album,
    x="album_submission_count",
    y="album_average_rank",
    color = TOP_COL_LOOKUP[COLUMN],
    size = "album_score",
    template="simple_white",
    labels={
        "album_average_rank": "Average Album Rank",
        "album_submission_count": "Album Submission Count",
        TOP_COL_LOOKUP[COLUMN] : "Top 10?"
    },
    custom_data=["Artist", "Album", "album_score"],
    opacity = 0.4,
    color_discrete_sequence=[APP_COLORS["accent"], APP_COLORS["standard"]]
)
figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata[0]}</b> - <i>%{customdata[1]}</i>",
            "",
            "Average Album Rank: %{y:.2f}",
            "Album Submission Count : %{x}",
            "Album Score: %{customdata[2]}",
            "<extra></extra>",
        ]
    ),
    # marker = {"size" : 12},
    selector=dict(mode="markers"),
)

figure.add_shape(
    type="line",
    x0=max_submissions/2, y0=0, x1=max_submissions/2, y1=10.5,
    line=dict(
        color=APP_COLORS["dark"],
        width=4,
        dash="dash",
    )
)
figure.add_shape(
    type="line",
    x0=1, y0=5.5, x1=max_submissions+1, y1=5.5,
    line=dict(
        color=APP_COLORS["dark"],
        width=4,
        dash="dash",
    )
)

figure.add_trace(go.Scatter(
    x=[2, 2, max_submissions-1, max_submissions-1],
    y=[0, 10.5, 0, 10.5],
    mode="text",
    name="Zones",
    text=["Hidden Gems", "Barely Made It", "Certified Bangers", "Easy Listening"],
    textposition="bottom center",
    textfont = {
        "color" : APP_COLORS["dark"],
        "size" : 13
    },
    showlegend = False,
    hoverinfo = "skip",
))

for row in range(len(albums_of_note)):
    album = albums_of_note.iloc[row,:]
    figure.add_annotation(
        x = int(album["album_submission_count"]),
        y = float(album["album_average_rank"]),
        text = album["Album"],
        font = {
        "color" : APP_COLORS["dim"],
        "size" : 9
    },

    )

figure["layout"]["yaxis"]["autorange"] = "reversed"

figure.show()


#### Album Lookups (individual Page)

* Album Voting Spread Stripplot
* Album Score Bargraph with adjacent scores
* User Album score/avg/count vs full data, top 10, and top 1

In [293]:
album_votes = AOTY.loc[AOTY["Album"] == USER_ALBUM]

avg_album_rank = album_votes["Rank"].mean()

figure = px.strip(
    data_frame=album_votes,
    x="Album",
    y="Rank",
    custom_data=["Artist"],
    labels={"Rank": "Rank on Voters Top 10 Lists", "Artist": ""},
    template="simple_white",
    width=300,
    color_discrete_sequence=[APP_COLORS["standard"]],
)

figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata}</b> - <i>%{x}</i>",
            "",
            f"<b>Total Submissions: {len(album_votes)}",
            "<extra></extra>",
        ]
    ),
)

# add average album rank
figure.add_shape(
    type="line",
    x0=-0.4,
    y0=avg_album_rank,
    x1=0.4,
    y1=avg_album_rank,
    name="Average Rank",
    line=dict(
        color=APP_COLORS["dark"],
        width=2,
        dash="dash",
    ),
)
figure.add_scatter(
    x = [USER_ALBUM],
    y = [avg_album_rank],
    text = "Average Rank",
    textposition="bottom right",
    textfont = {
        "color" : APP_COLORS["dark"],
        "size" : 8
    },
    showlegend = False,
    hoverinfo = "y",
    mode="text")



figure.update_layout(hovermode="y", xaxis={"visible": False, "showticklabels": False})


figure["layout"]["yaxis"]["autorange"] = "reversed"

figure


In [191]:
# TODO: Write tests for get_competing_albums and put into datafunc
# from data.datafunc import get_competing_albums
def get_competing_albums(df: pd.DataFrame, USER_ALBUM : str) -> pd.DataFrame:
    """
    Returns a smaller subset of the nearby ranked albums
    based on the rank of the user album passed in, with
    an additional column to note which album is selected.

    Parameters
    ----------
    df : pd.DataFrame
        The AOTY by album df in a meaningful order.
    USER_ALBUM : str
        Title of album to lookup

    Returns
    -------
    pd.DataFrame
        A dataframe of 5 rows surrounding (or at edge) of
        user selected album.
    """
    new_df = df.copy()
    
    # Get 5 albums, ideally 2 on either side (adjusting for endpoints).
    user_album_rank = new_df.loc[new_df["Album"] == USER_ALBUM].index[0]
    if user_album_rank == 0:
        rank_range = range(user_album_rank, user_album_rank + 5)
    elif user_album_rank == 1:
        rank_range = range(user_album_rank - 1, user_album_rank + 4)
    elif user_album_rank == len(new_df) - 2:
        rank_range = range(user_album_rank - 3, user_album_rank + 2)
    elif user_album_rank == len(new_df) - 1:
        rank_range = range(user_album_rank - 4, user_album_rank + 1)
    else:
        rank_range = range(user_album_rank - 2, user_album_rank + 3)

    new_df = new_df.iloc[rank_range, :]
    
    new_df["user_album"] = new_df["Album"].apply(
        lambda album:
            True if album == USER_ALBUM
            else False
    )
    return new_df


In [292]:
nearby_df = get_competing_albums(AOTY_by_album, USER_ALBUM)

figure = px.bar(
    data_frame=nearby_df,
    x="Album",
    y=COLUMN,
    color="user_album",
    custom_data=[
        nearby_df.index + 1,
        "Artist",
        "album_score",
        "album_submission_count",
    ],
    labels={
        "album_score": "Album Score",
        "album_submission_count": "Album Submission Count",
    },
    template="simple_white",
    text_auto=True,
    color_discrete_map={True: APP_COLORS["accent"], False: APP_COLORS["standard"]},
)

figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>Rank %{customdata[0]}:</b>",
            "%{customdata[1]} - <i>%{x}</i>",
            "",
            "Total Score: %{customdata[2]}",
            "Submission Count: %{customdata[3]}",
            "<extra></extra>",
        ]
    )
)

figure.update_layout(
    hovermode="closest", showlegend=False, xaxis_categoryorder="total descending"
)


In [193]:
AOTY_by_album

user_and_top_album = AOTY_by_album.loc[
    (AOTY_by_album["Album"] == USER_ALBUM)
    | (AOTY_by_album["album_score"] == AOTY_by_album["album_score"].max())
]

album_summary = pd.DataFrame(AOTY_by_album.mean()).transpose()
top_10_album_summary = pd.DataFrame(top_10_albums.mean()).transpose()

top_10_album_summary["Album"] = "Top 10 Average"
album_summary["Album"] = "Overall Average"

top_10_album_summary["Artist"] = "Various"
album_summary["Artist"] = "Various"

# compare_df = pd.concat([user_and_top_album, top_10_album_summary, album_summary])
compare_df = user_and_top_album

compare_df["user_album"] = compare_df["Album"].apply(
    lambda album: True if album == USER_ALBUM else False
)

compare_df


,Artist,Album,album_score,album_average_rank,album_submission_count,top_10_score_album,top_10_count_album,unique_album_submission,artist_album_release_count,multi_album_artist,user_album
0,Silk Sonic,An Evening With Silk Sonic,171,4.160000,25,True,True,False,1,False,False
14,Turnstile,Glow On,55,4.888889,9,False,False,False,1,False,True


In [285]:
figure = px.bar(
    data_frame=compare_df,
    x="Album",
    y=COLUMN,
    color="user_album",
    custom_data=["Artist", "album_score", "album_submission_count"],
    labels={"album_score": "Overall Album Score", "album_submission_count": "Album Submission Count", "Album": ""},
    template="simple_white",
    text_auto=True,
    color_discrete_map={True: APP_COLORS["accent"], False: APP_COLORS["standard"]},
    width = 400
)

figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata[0]}</b> - <i>%{x}</i>",
            "",
            "<extra></extra>",
        ]
    )
)

# overall average
overall_average = AOTY_by_album[COLUMN].mean()
figure.add_hrect(
    y0=overall_average - 2,
    y1=overall_average + 2,
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text="Overall Average",
    annotation_font={"size": 8},
    annotation_position="outside right",
    annotation_font_color=APP_COLORS["dark"],
)

# top 10 average
top_10_average = top_10_albums[COLUMN].mean()
figure.add_hrect(
    y0=top_10_average - 2,
    y1=top_10_average + 2,
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text="Top 10 Average",
    annotation_font={"size": 8},
    annotation_position="outside right",
    annotation_font_color=APP_COLORS["dark"],
)

# Add invisible points for hover
figure.add_scatter(
    x = [USER_ALBUM, USER_ALBUM],
    y = [overall_average, top_10_average],
    showlegend = False,
    hoverinfo = "y",
    mode="none")



figure.update_layout(hovermode="y", showlegend=False, xaxis_categoryorder="trace")


In [291]:
figure = px.bar(
    data_frame=compare_df,
    x="Album",
    y="album_average_rank",
    color="user_album",
    custom_data=["Artist", "album_score", "album_submission_count"],
    labels={"album_average_rank": "Average Album Voter Rank", "Album": ""},
    template="simple_white",
    text_auto=True,
    color_discrete_map={True: APP_COLORS["accent"], False: APP_COLORS["standard"]},
    width = 400
)

figure.update_traces(
    hovertemplate="<br>".join(
        [
            "<b>%{customdata[0]}</b> - <i>%{x}</i>",
            "",
            "<extra></extra>",
        ]
    )
)

# overall average
overall_average = AOTY_by_album["album_average_rank"].mean()
figure.add_hrect(
    y0=overall_average - 0.05,
    y1=overall_average + 0.05,
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text="Overall Average",
    annotation_font={"size": 8},
    annotation_position="outside right",
    annotation_font_color=APP_COLORS["dark"],
)

# top 10 average
top_10_average = top_10_albums["album_average_rank"].mean()
figure.add_hrect(
    y0=top_10_average - 0.05,
    y1=top_10_average + 0.05,
    opacity=0.3,
    fillcolor=APP_COLORS["dark"],
    annotation_text="Top 10 Average",
    annotation_font={"size": 8},
    annotation_position="outside right",
    annotation_font_color=APP_COLORS["dark"],
)

# Add invisible points for hover
figure.add_scatter(
    x = [USER_ALBUM, USER_ALBUM],
    y = [overall_average, top_10_average],
    showlegend = False,
    hoverinfo = "y",
    mode="none")



figure.update_layout(hovermode="y", showlegend=False, xaxis_categoryorder="trace")
